Social network metrics

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [23]:
social = pd.read_csv('degree.csv',encoding='utf-8')
social.head()

,Vertices,Degree,Clustering,Class
0,n85004928,1,0.000000,0
1,n85004929,7,0.261905,3
2,n85008101,5,0.200000,2
3,n85021444,5,0.200000,2
4,n85004930,1,0.000000,0


In [24]:
social.drop(['Vertices'],axis = 1,inplace = True)


In [25]:
print(social.describe())
total_sample=social.shape[0]
print(total_sample)

              Degree     Clustering          Class
count  535890.000000  535890.000000  535890.000000
mean        2.775999       0.004179       0.616186
std        52.062979       0.040359       0.994757
min         1.000000       0.000000       0.000000
25%         1.000000       0.000000       0.000000
50%         1.000000       0.000000       0.000000
75%         2.000000       0.000000       1.000000
max     35358.000000       1.000000       4.000000
535890


In [26]:
X = social.iloc[:,0:2].values
Y = social['Class'].values
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size =.25)
X_traino=X_train
X_testo=X_test

In [27]:
X_train.shape

(401917, 2)

In [28]:
X_test.shape

(133973, 2)

In [29]:
stdscaler = StandardScaler()
X_train = stdscaler.fit_transform(X_train)
X_test = stdscaler.transform(X_test)

In [30]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train,Y_train)

GaussianNB()

In [31]:
y_pred = clf.predict(X_test)
y_predproba=clf.predict_proba(X_test)
y_predprobatrain=clf.predict_proba(X_train)

In [32]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(Y_test, y_pred)


In [33]:
print(cm)

[[84595     0     0     0     0]
 [    0 29589     0     0     0]
 [    0   170 10161     0     0]
 [    0     0    53  5643     0]
 [    0     0     0   500  3262]]


In [34]:
print(accuracy_score(Y_test, y_pred))

0.9946033902353459


In [39]:
from sklearn.metrics import classification_report
print(f'Classification Report: \n{classification_report(Y_test, y_pred,digits=3)}')

Classification Report: 
              precision    recall  f1-score   support

           0      1.000     1.000     1.000     84595
           1      0.994     1.000     0.997     29589
           2      0.995     0.984     0.989     10331
           3      0.919     0.991     0.953      5696
           4      1.000     0.867     0.929      3762

    accuracy                          0.995    133973
   macro avg      0.982     0.968     0.974    133973
weighted avg      0.995     0.995     0.995    133973



In [36]:
output_column=social.columns.values.tolist()
print(output_column)

['Degree', 'Clustering', 'Class']


In [37]:
import openpyxl
wb=openpyxl.Workbook()
s1=wb.active

s1['A1']=output_column[0]
s1['B1']=output_column[1]
s1['C1']='posterior probability'
s1['D1']='class'
wb.save('outputsocial.xlsx')

In [38]:
traintotal=X_train.shape[0]
yp=[]
xp=[]
nclass=4
c3class=0

workbook=openpyxl.load_workbook('outputsocial.xlsx')
sheet=workbook.active
for i in range(traintotal):
    if(Y_train[i]==nclass):
        c3class += 1
        yp.append(y_predprobatrain[i,nclass])
        xp.append(X_traino[i,:])

testtotal=X_test.shape[0]

for i in range(testtotal):
    if(Y_test[i]==nclass):
        c3class += 1
        yp.append(y_predproba[i,nclass])
        xp.append(X_testo[i,:])

print(c3class/total_sample)
C=75
ci_upper=np.percentile(yp,100-(100-C)*0.5)
ci_lower=np.percentile(yp,(100-C)*0.5)
ppc=c3class/total_sample
cxlower=35.01
j=2
for k in range(len(yp)):
    if(ci_lower<=yp[k]<=ci_upper):
        if(yp[k]*1./ppc>=cxlower):
            sheet.cell(j,1).value=xp[k][0]
            sheet.cell(j,2).value=xp[k][1]
            sheet.cell(j,3).value=yp[k]
            sheet.cell(j,4).value=nclass
            j+=1
        
workbook.save('outputsocial.xlsx')

0.02855809960999459
